In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, Dataset

from torchvision.transforms import ToTensor

from tqdm import tqdm, trange

from zipfile import ZipFile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 

from numpy.typing import NDArray
from typing import Tuple

np.random.seed(0)
torch.manual_seed(0)

RSEED = 0
BATCH = 32

In [63]:
with ZipFile("Colorectal_Histology.zip", "r") as zObject:
    zObject.extractall(path=".")

In [64]:
df = pd.read_csv("hmnist_64_64_L.csv")
X = df.drop('label', axis=1)
X = X.to_numpy()
y = df.label.to_numpy()


In [65]:
type(y)

numpy.ndarray

In [66]:
X

array([[134,  99, 119, ..., 126, 140, 195],
       [ 55,  64,  74, ...,  65,  68,  69],
       [114, 116, 136, ..., 129,  89, 122],
       ...,
       [ 30,  29,  37, ..., 195,  83,  45],
       [105, 115, 108, ...,  77,  81,  85],
       [ 60,  77,  84, ...,  90,  64,  67]], shape=(5000, 4096))

In [67]:
class histo(Dataset):
    def __init__(self, X:NDArray[np.int8], y:NDArray[np.int8])->None:
        self.X = X.astype(np.float32)/255
        if y is None:
            self.y = None
        else:
            self.y = y

    def __len__(self)->int:
        return len(self.X)
    
    def __getitem__(self, idx:int)->int:
        return self.X[idx], self.y[idx]
    
    @staticmethod
    def create_split(X, y, train_fraction:float, val_fraction:float, test_fraction:float)->Tuple[Dataset, Dataset, Dataset]:
        assert train_fraction + val_fraction + test_fraction <= 1

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_fraction)

        train_data = histo(X_train, y_train)
        val_data = histo(X_val, y_val)

        return train_data, val_data


In [68]:
train_data, val_data = histo.create_split(X, y, 0.5, 0.5,0.0)

In [70]:
train_data[25]

(array([0.28627452, 0.30588236, 0.40784314, ..., 0.30980393, 0.29411766,
        0.29411766], shape=(4096,), dtype=float32),
 np.int64(5))

In [71]:
def  main():


    train_loader = DataLoader(train_data, batch_size=BATCH, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=BATCH, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ViTModel((1, 28, 28), n_patches=16, n_blocks=2, hidden_d=8, n_heads=2, out_d=10).to(device)
    num_epochs = 4
    learn_rate = 0.005

    optimizer = Adam(model.parameters(), lr=learn_rate)
    criterion = CrossEntropyLoss()

    for epoch in trange(num_epochs, desc="Training"):
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch +1} in training", leave=False):
            X, y = batch
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = criterion(y_pred, y)

            train_loss += loss.detach().cpu().item()/len(train_loader)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{num_epochs} loss: {train_loss:.3f}")

        with torch.no_grad():
            correct, total = 0,  0

            test_loss = 0.0

            for batch in tqdm(val_loader, desc="Testing"):
                X, y = batch
                X, y = X.to(device), y.to(device)

                y_pred = model(X)

                loss = criterion(y_pred, y)

                test_loss += loss.detach().cpu().item()/len(test_loader)

                correct += torch.sum(torch.argmax(y_pred, dim=1)==y).detach().cpu().item()
                total += len(X)
            print(f"Test loss: {test_loss:.3f}")
            print(f"Test accuracy: {correct/total*100:.3f}")



    